In [1]:
#imports
import pandas as pd
import numpy as np
import tensorflow as tf
tf.config.run_functions_eagerly(True)

In [2]:
dataset = np.load("proyecto_training_data.npy")
dataset = dataset[~np.isnan(dataset).any(axis=1), :] #Dataset contiene nans, los ignoro en la implementacion
fields = ["SalePrice", "OverallQual", "1stFlrSF", "TotRmsAbvGrd", "YearBuilt", "LotFrontage"]
print(np.vstack((np.array(fields), dataset)))

[['SalePrice' 'OverallQual' '1stFlrSF' 'TotRmsAbvGrd' 'YearBuilt'
  'LotFrontage']
 ['208500.0' '7.0' '856.0' '8.0' '2003.0' '65.0']
 ['181500.0' '6.0' '1262.0' '6.0' '1976.0' '80.0']
 ...
 ['266500.0' '7.0' '1188.0' '9.0' '1941.0' '66.0']
 ['142125.0' '5.0' '1078.0' '5.0' '1950.0' '68.0']
 ['147500.0' '5.0' '1256.0' '6.0' '1965.0' '75.0']]


In [3]:
class LinearModel:
    def __init__(self, batch_size = 16, lr=0.01, epochs = 1000):
        self.logdir = f'logs\\lm_bs={batch_size}_lr={lr}_epochs={epochs}_bs={batch_size}'
        self.writer = tf.summary.create_file_writer(self.logdir)
        self.m = tf.Variable(initial_value=0.0, name="slope")
        self.b = tf.Variable(initial_value=0.0, name="intercept")
        tf.summary.trace_on(graph=True, profiler=False)
        
        self.batch_size = batch_size 
        self.lr = lr 
        self.epochs = epochs
        
    def h(self, m, b, x):
        y = tf.cast(m, tf.float64)*tf.cast(x, tf.float64) + tf.cast(b, tf.float64)
        return y
    
    def get_params(self):
        return((self.m, self.b))
    
    def error(self, y,y_pred):
        return 1/2*tf.reduce_mean(tf.math.square(y - y_pred), name="MSE_Calc")

    def __call__(self,x):
        return self.h(self.m, self.b, x)
    
    @tf.function
    def train(self, x, y):
        batch_size = self.batch_size
        lr = self.lr 
        epochs = self.epochs
        
        iterations = int(len(y)/batch_size)
        step = 0
        with self.writer.as_default():
            for epoch in range(epochs):
                for i in range(iterations): 
                    batch_start = i*batch_size
                    batch_end = batch_start + batch_size
                    x_mb = tf.reshape(x[batch_start:batch_end], [-1,1])
                    y_mb = tf.reshape(y[batch_start:batch_end], [-1,1])
                        
                        
                    with tf.GradientTape() as grad_tape:
                        grad_tape.watch(self.b)
                        grad_tape.watch(self.m)

                        y_pred = self.h(self.m, self.b, x_mb)

                        error = self.error(y_mb, y_pred)

                    # calcular el gradiente de la funcion de costo respecto de los parametros
                    grad_m,grad_b = grad_tape.gradient(error,[self.m, self.b])

                    # actualizar los parametros dando un paso en direccion contraria al gradiente
                    self.m.assign(self.m - lr*grad_m)
                    self.b.assign(self.b - lr*grad_b)
                    tf.summary.scalar('MSE', error, step=step)
                    step += 1
        final_params = (self.m, self.b)
        #self.writer.flush()

        return final_params

In [4]:
writer = tf.summary.create_file_writer("logs\\modelgraph")
model1 = LinearModel(epochs = 50, lr = 0.0001, batch_size=200)
tf.summary.trace_on(graph=True)
model1.train(dataset[:,1], dataset[:,0])
with writer.as_default():
    tf.summary.trace_export(
                                  name="my_func_trace",
                                  step=0,
                                  profiler_outdir="logs\\modelgraph")

In [5]:
print(model1(1))

tf.Tensor(23987.783203125, shape=(), dtype=float64)


In [6]:
model1.get_params()

(<tf.Variable 'slope:0' shape=() dtype=float32, numpy=20914.29>,
 <tf.Variable 'intercept:0' shape=() dtype=float32, numpy=3073.4941>)

# Experimentos

Hipotesis: Un LR alto causara una reduccion de error mas rapidamente pero causara divergencia mas tardia

In [7]:
import itertools
lrs = [0.1, 0.01, 0.001]
epochs = [100, 1000]
batch_sizes = [100, 500]

for lr, epoch, batch_size in list(itertools.product(lrs, epochs, batch_sizes)):
    model = LinearModel(epochs = epoch, lr = lr, batch_size = batch_size)
    model.train(dataset[:,1], dataset[:,0])

![alt text](MSEs.png "MSEs")

In [ ]:
El error tiende a ser mas bajo cuando Batch Size = 100, lr+0.001, e